In [16]:
import requests
import json

# url = "https://www.thesportsdb.com/api/v1/json/3/eventsseason.php?id=4481&s=2023-2024" # matches of the season 2023-2024 for ligue 1
url = "https://www.thesportsdb.com/api/v1/json/3/eventsround.php?id=4481&r=1&s=2023-2024" # For league 1 the round correspond to the number of the day in the competition, there is 38 days in a season of ligue 1 
# url = "https://www.thesportsdb.com/api/v1/json/3/search_all_leagues.php?c=Europe&s=Soccer"
api_key = "3"

headers = {
    "X-API-KEY": f"{api_key}",
    "Content-Type": "application/json"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    print(json.dumps(response.json(), indent=4))
else:
    print(f"Request failed with status code: {response.status_code}")

{
    "events": [
        {
            "idEvent": "1890063",
            "idAPIfootball": "1126336",
            "strEvent": "Union Saint-Gilloise vs Toulouse",
            "strEventAlternate": "Toulouse @ St. Gilloise",
            "strFilename": "UEFA Europa League 2023-09-21 Union Saint-Gilloise vs Toulouse",
            "strSport": "Soccer",
            "idLeague": "4481",
            "strLeague": "UEFA Europa League",
            "strLeagueBadge": "https://www.thesportsdb.com/images/media/league/badge/mlsr7d1718774547.png",
            "strSeason": "2023-2024",
            "strDescriptionEN": "",
            "strHomeTeam": "St. Gilloise",
            "strAwayTeam": "Toulouse",
            "intHomeScore": "1",
            "intRound": "1",
            "intAwayScore": "1",
            "intSpectators": null,
            "strOfficial": "",
            "strTimestamp": "2023-09-21T16:45:00",
            "dateEvent": "2023-09-21",
            "dateEventLocal": "2023-09-21",
            "

In [16]:
import requests
import json
import pandas as pd
import time

def create_match_dataset_multiple_seasons_ligue_1(league_id=4334, start_season, end_season):
    """
    Crée un dataset Pandas contenant les informations des matchs de plusieurs saisons de football pour l'Olympique Lyonnais
    en utilisant l'endpoint eventsround.php et en respectant la limite de 100 requêtes par minute.

    Args:
        league_id (int): L'identifiant de la ligue dans l'API (e.g., 4334 pour la Ligue 1).
        start_season (int): L'année de début de la première saison (e.g., 2012).
        end_season (int): L'année de début de la dernière saison (e.g., 2024).

    Returns:
        pandas.DataFrame: Un DataFrame contenant les informations des matchs de l'OL, ou None si une des requêtes a échoué.
    """

    all_data = []
    ol_team_id = "133713"  # ID de l'Olympique Lyonnais
    api_key = "3"
    headers = {
        "X-API-KEY": f"{api_key}",
        "Content-Type": "application/json"
    }
    request_count = 0
    start_time = time.time()

    for year in range(start_season, end_season + 1):
        season = f"{year}-{year + 1}"
        print(f"Récupération des données pour la saison {season}...")

        for round_number in range(1, 39):  # Boucle sur les rounds (journées) de 1 à 38
            # Gestion de la limite de requêtes par minute
            if request_count >= 100:
                elapsed_time = time.time() - start_time
                if elapsed_time < 120:
                    sleep_time = 120 - elapsed_time
                    print(f"Limite de requêtes atteinte. Attente de {sleep_time:.2f} secondes...")
                    time.sleep(sleep_time)
                request_count = 0
                start_time = time.time()

            print(f"  Récupération des données pour la journée {round_number}...")
            url = f"https://www.thesportsdb.com/api/v1/json/3/eventsround.php?id={league_id}&r={round_number}&s={season}"

            response = requests.get(url, headers=headers)
            request_count += 1

            if response.status_code == 200:
                data = response.json()
                if "events" in data and data["events"]:
                    events = data["events"]
                    for event in events:
                        # Filtrer les matchs de l'OL
                        if event["idHomeTeam"] == ol_team_id or event["idAwayTeam"] == ol_team_id:
                            all_data.append({
                                "idEvent": event["idEvent"],
                                "strEvent": event["strEvent"],
                                "strSeason": event["strSeason"],
                                "intHomeScore": event["intHomeScore"],
                                "intAwayScore": event["intAwayScore"],
                                "idHomeTeam": event["idHomeTeam"],
                                "idAwayTeam": event["idAwayTeam"],
                                "strTimestamp": event["strTimestamp"],
                                "dateEvent": event["dateEvent"],
                                "strTime": event["strTime"],
                                "strLeague": event["strLeague"],
                                "idLeague": event["idLeague"],
                                "strCountry": event["strCountry"],
                            })
                else:
                    print(f"    Aucun événement trouvé pour la ligue {league_id}, la saison {season} et la journée {round_number}.")
            else:
                print(f"    Request failed with status code: {response.status_code} pour la saison {season}, journée {round_number}")
                return None  # Arrête la fonction si une requête échoue

    return pd.DataFrame(all_data)

# Exemple d'utilisation:
league_id = 4334  # Ligue 1
start_season = 2012
end_season = 2024
df = create_match_dataset_multiple_seasons_ligue_1(league_id, start_season, end_season)

if df is not None:
    print(df.head())
    print(df.shape)

Récupération des données pour la saison 2012-2013...
  Récupération des données pour la journée 1...
  Récupération des données pour la journée 2...
  Récupération des données pour la journée 3...
  Récupération des données pour la journée 4...
  Récupération des données pour la journée 5...
  Récupération des données pour la journée 6...
  Récupération des données pour la journée 7...
  Récupération des données pour la journée 8...
  Récupération des données pour la journée 9...
  Récupération des données pour la journée 10...
  Récupération des données pour la journée 11...
  Récupération des données pour la journée 12...
  Récupération des données pour la journée 13...
  Récupération des données pour la journée 14...
  Récupération des données pour la journée 15...
  Récupération des données pour la journée 16...
  Récupération des données pour la journée 17...
  Récupération des données pour la journée 18...
  Récupération des données pour la journée 19...
  Récupération des donnée

In [17]:
df.head(100000)

,idEvent,strEvent,strSeason,intHomeScore,intAwayScore,idHomeTeam,idAwayTeam,strTimestamp,dateEvent,strTime,strLeague,idLeague,strCountry
0,460132,Rennes vs Lyon,2012-2013,0,1,133719,133713,None,2012-08-11,18:00:00+00:00,French Ligue 1,4334,None
1,460141,Lyon vs Troyes,2012-2013,4,1,133713,134789,None,2012-08-18,14:00:00+00:00,French Ligue 1,4334,None
2,460150,Evian Thonon Gaillard vs Lyon,2012-2013,1,1,133705,133713,None,2012-08-24,17:45:00+00:00,French Ligue 1,4334,None
3,460161,Lyon vs Valenciennes,2012-2013,3,2,133713,133706,None,2012-09-01,14:00:00+00:00,French Ligue 1,4334,None
4,460178,Lyon vs Ajaccio,2012-2013,2,0,133713,133702,None,2012-09-16,14:00:00+00:00,French Ligue 1,4334,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,2076484,St Etienne vs Lyon,2024-2025,None,None,133717,133713,2025-04-20T00:00:00,2025-04-20,00:00:00,French Ligue 1,4334,France
469,2076501,Lyon vs Rennes,2024-2025,None,None,133713,133719,2025-04-27T00:00:00,2025-04-27,00:00:00,French Ligue 1,4334,France
470,2076502,Lyon vs Lens,2024-2025,None,None,133713,133822,2025-05-04T00:00:00,2025-05-04,00:00:00,French Ligue 1,4334,France
471,2076519,Monaco vs Lyon,2024-2025,None,None,133823,133713,2025-05-11T00:00:00,2025-05-11,00:00:00,French Ligue 1,4334,Monaco


In [17]:
import pandas as pd

# df.to_pickle('raw_data_ol_match_ligue_1.pkl')
df = pd.read_pickle('raw_data_ol_match_ligue_1.pkl')

In [18]:
import requests
import json
import pandas as pd
import time

def add_coupe_de_france_match_dataset_multiple_seasons(df_ligue1, start_season, end_season):
    """
    Ajoute au dataset existant les informations des matchs de Coupe de France de football pour l'Olympique Lyonnais
    en utilisant l'endpoint eventsround.php et en respectant la limite de 100 requêtes par minute.

    Args:
        df_ligue1 (pandas.DataFrame): Le DataFrame contenant les informations des matchs de Ligue 1.
        start_season (int): L'année de début de la première saison (e.g., 2012).
        end_season (int): L'année de début de la dernière saison (e.g., 2024).

    Returns:
        pandas.DataFrame: Un DataFrame contenant les informations des matchs de l'OL en Ligue 1 et Coupe de France,
                         ou None si une des requêtes a échoué.
    """

    all_coupe_de_france_data = []
    ol_team_id = "133713"  # ID de l'Olympique Lyonnais
    coupe_de_france_league_id = 4484
    api_key = "3"
    headers = {
        "X-API-KEY": f"{api_key}",
        "Content-Type": "application/json"
    }
    request_count = 0
    start_time = time.time()
    coupe_de_france_rounds = [200, 150, 125, 16, 32, 64]

    for year in range(start_season, end_season + 1):
        season = f"{year}-{year + 1}"
        print(f"Récupération des données de la Coupe de France pour la saison {season}...")

        for round_number in coupe_de_france_rounds:
            # Gestion de la limite de requêtes par minute
            if request_count >= 100:
                elapsed_time = time.time() - start_time
                if elapsed_time < 100:  # Correction ici: la limite est de 100 requêtes par minute, donc on attend jusqu'à 60 secondes
                    sleep_time = 100 - elapsed_time
                    print(f"Limite de requêtes atteinte. Attente de {sleep_time:.2f} secondes...")
                    time.sleep(sleep_time)
                request_count = 0
                start_time = time.time()

            print(f"  Récupération des données pour le tour {round_number}...")
            url = f"https://www.thesportsdb.com/api/v1/json/3/eventsround.php?id={coupe_de_france_league_id}&r={round_number}&s={season}"

            response = requests.get(url, headers=headers)
            request_count += 1

            if response.status_code == 200:
                data = response.json()
                if "events" in data and data["events"]:
                    events = data["events"]
                    for event in events:
                        # Filtrer les matchs de l'OL
                        if event["idHomeTeam"] == ol_team_id or event["idAwayTeam"] == ol_team_id:
                            all_coupe_de_france_data.append({
                                "idEvent": event["idEvent"],
                                "strEvent": event["strEvent"],
                                "strSeason": event["strSeason"],
                                "intHomeScore": event.get("intHomeScore"), # Use .get() to handle potential missing keys
                                "intAwayScore": event.get("intAwayScore"), # Use .get() to handle potential missing keys
                                "idHomeTeam": event["idHomeTeam"],
                                "idAwayTeam": event["idAwayTeam"],
                                "strTimestamp": event.get("strTimestamp"), # Use .get() to handle potential missing keys
                                "dateEvent": event["dateEvent"],
                                "strTime": event["strTime"],
                                "strLeague": event["strLeague"],
                                "idLeague": event["idLeague"],
                                "strCountry": event.get("strCountry"), # Use .get() to handle potential missing keys
                            })
                else:
                    print(f"    Aucun événement trouvé pour la Coupe de France, la saison {season} et le tour {round_number}.")
            else:
                print(f"    Request failed with status code: {response.status_code} pour la saison {season}, tour {round_number}")
                return None  # Arrête la fonction si une requête échoue

    df_coupe_de_france = pd.DataFrame(all_coupe_de_france_data)

    if df_coupe_de_france.empty:
        print("Aucun match de Coupe de France trouvé pour l'OL durant les saisons spécifiées.")
        return df_ligue1  # Retourne le DataFrame de Ligue 1 si aucun match de Coupe de France n'est trouvé
    else:
        # Combiner les DataFrames
        combined_df = pd.concat([df_ligue1, df_coupe_de_france], ignore_index=True)
        return combined_df

# Exemple d'utilisation (en supposant que vous avez déjà exécuté la première fonction):
league_id = 4334  # Ligue 1
start_season = 2019
end_season = 2024
df_ligue1 = add_coupe_de_france_match_dataset_multiple_seasons(df, start_season, end_season)

Récupération des données de la Coupe de France pour la saison 2019-2020...
  Récupération des données pour le tour 200...
  Récupération des données pour le tour 150...
  Récupération des données pour le tour 125...
  Récupération des données pour le tour 16...
    Aucun événement trouvé pour la Coupe de France, la saison 2019-2020 et le tour 16.
  Récupération des données pour le tour 32...
    Aucun événement trouvé pour la Coupe de France, la saison 2019-2020 et le tour 32.
  Récupération des données pour le tour 64...
    Aucun événement trouvé pour la Coupe de France, la saison 2019-2020 et le tour 64.
Récupération des données de la Coupe de France pour la saison 2020-2021...
  Récupération des données pour le tour 200...
  Récupération des données pour le tour 150...
  Récupération des données pour le tour 125...
  Récupération des données pour le tour 16...
  Récupération des données pour le tour 32...
  Récupération des données pour le tour 64...
Récupération des données de la 

In [19]:
df_ligue1.head(100000)

,idEvent,strEvent,strSeason,intHomeScore,intAwayScore,idHomeTeam,idAwayTeam,strTimestamp,dateEvent,strTime,strLeague,idLeague,strCountry
0,460132,Rennes vs Lyon,2012-2013,0,1,133719,133713,None,2012-08-11,18:00:00+00:00,French Ligue 1,4334,None
1,460141,Lyon vs Troyes,2012-2013,4,1,133713,134789,None,2012-08-18,14:00:00+00:00,French Ligue 1,4334,None
2,460150,Evian Thonon Gaillard vs Lyon,2012-2013,1,1,133705,133713,None,2012-08-24,17:45:00+00:00,French Ligue 1,4334,None
3,460161,Lyon vs Valenciennes,2012-2013,3,2,133713,133706,None,2012-09-01,14:00:00+00:00,French Ligue 1,4334,None
4,460178,Lyon vs Ajaccio,2012-2013,2,0,133713,133702,None,2012-09-16,14:00:00+00:00,French Ligue 1,4334,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,1936419,Lyon vs Lille,2023-2024,2,1,133713,133711,2024-02-07T17:30:00,2024-02-07,17:30:00,Coupe de France,4484,France
487,1931724,Bergerac Périgord vs Lyon,2023-2024,1,2,144767,133713,2024-01-19T19:45:00,2024-01-19,19:45:00,Coupe de France,4484,France
488,1922516,CA Pontarlier vs Lyon,2023-2024,0,3,142610,133713,2024-01-07T13:30:00,2024-01-07,13:30:00,Coupe de France,4484,France
489,2190050,Bourgoin-Jallieu vs Lyon,2024-2025,None,None,147564,133713,2025-01-15T17:00:00,2025-01-15,17:00:00,Coupe de France,4484,France


In [20]:
import requests
import json
import pandas as pd
import time

def add_champions_league_match_dataset_multiple_seasons(df_ligue1, start_season, end_season):
    """
    Adds Champions League match information for Olympique Lyonnais to an existing dataset
    using the eventsround.php endpoint and respecting the limit of 100 requests per minute.

    Args:
        df_ligue1 (pandas.DataFrame): The DataFrame containing Ligue 1 match information.
        start_season (int): The start year of the first season (e.g., 2012).
        end_season (int): The start year of the last season (e.g., 2024).

    Returns:
        pandas.DataFrame: A DataFrame containing OL's match information in Ligue 1 and Champions League,
                         or None if any of the requests failed.
    """

    all_champions_league_data = []
    ol_team_id = "133713"  # ID of Olympique Lyonnais
    champions_league_league_id = 4480
    api_key = "3"
    headers = {
        "X-API-KEY": f"{api_key}",
        "Content-Type": "application/json"
    }
    request_count = 0
    start_time = time.time()
    champions_league_rounds = [200, 150, 125, 16, 1, 2, 3, 4, 5, 6, 7, 8]

    for year in range(start_season, end_season + 1):
        season = f"{year}-{year + 1}"
        print(f"Retrieving Champions League data for the season {season}...")

        for round_number in champions_league_rounds:
            # Managing the request limit per minute
            if request_count >= 100:
                elapsed_time = time.time() - start_time
                if elapsed_time < 100:
                    sleep_time = 100 - elapsed_time
                    print(f"Request limit reached. Waiting for {sleep_time:.2f} seconds...")
                    time.sleep(sleep_time)
                request_count = 0
                start_time = time.time()

            print(f"  Retrieving data for round {round_number}...")
            url = f"https://www.thesportsdb.com/api/v1/json/3/eventsround.php?id={champions_league_league_id}&r={round_number}&s={season}"

            response = requests.get(url, headers=headers)
            request_count += 1

            if response.status_code == 200:
                data = response.json()
                if "events" in data and data["events"]:
                    events = data["events"]
                    for event in events:
                        # Filter OL matches
                        if event["idHomeTeam"] == ol_team_id or event["idAwayTeam"] == ol_team_id:
                            all_champions_league_data.append({
                                "idEvent": event["idEvent"],
                                "strEvent": event["strEvent"],
                                "strSeason": event["strSeason"],
                                "intHomeScore": event.get("intHomeScore"),
                                "intAwayScore": event.get("intAwayScore"),
                                "idHomeTeam": event["idHomeTeam"],
                                "idAwayTeam": event["idAwayTeam"],
                                "strTimestamp": event.get("strTimestamp"),
                                "dateEvent": event["dateEvent"],
                                "strTime": event["strTime"],
                                "strLeague": event["strLeague"],
                                "idLeague": event["idLeague"],
                                "strCountry": event.get("strCountry"),
                            })
                else:
                    print(f"    No events found for the Champions League, season {season}, and round {round_number}.")
            else:
                print(f"    Request failed with status code: {response.status_code} for season {season}, round {round_number}")
                return None  # Stop the function if a request fails

    df_champions_league = pd.DataFrame(all_champions_league_data)

    if df_champions_league.empty:
        print("No Champions League matches found for OL during the specified seasons.")
        return df_ligue1  # Return the Ligue 1 DataFrame if no Champions League matches are found
    else:
        # Combine the DataFrames
        combined_df = pd.concat([df_ligue1, df_champions_league], ignore_index=True)
        return combined_df
    
# Exemple d'utilisation (en supposant que vous avez déjà exécuté la première fonction):
start_season = 2000
end_season = 2024
df_ligue1 = add_champions_league_match_dataset_multiple_seasons(df_ligue1, start_season, end_season)

Retrieving Champions League data for the season 2000-2001...
  Retrieving data for round 200...
  Retrieving data for round 150...
    No events found for the Champions League, season 2000-2001, and round 150.
  Retrieving data for round 125...
    No events found for the Champions League, season 2000-2001, and round 125.
  Retrieving data for round 16...
    No events found for the Champions League, season 2000-2001, and round 16.
  Retrieving data for round 1...
    No events found for the Champions League, season 2000-2001, and round 1.
  Retrieving data for round 2...
    No events found for the Champions League, season 2000-2001, and round 2.
  Retrieving data for round 3...
    No events found for the Champions League, season 2000-2001, and round 3.
  Retrieving data for round 4...
    No events found for the Champions League, season 2000-2001, and round 4.
  Retrieving data for round 5...
    No events found for the Champions League, season 2000-2001, and round 5.
  Retrieving d

In [21]:
df_ligue1.head(100000)

,idEvent,strEvent,strSeason,intHomeScore,intAwayScore,idHomeTeam,idAwayTeam,strTimestamp,dateEvent,strTime,strLeague,idLeague,strCountry
0,460132,Rennes vs Lyon,2012-2013,0,1,133719,133713,None,2012-08-11,18:00:00+00:00,French Ligue 1,4334,None
1,460141,Lyon vs Troyes,2012-2013,4,1,133713,134789,None,2012-08-18,14:00:00+00:00,French Ligue 1,4334,None
2,460150,Evian Thonon Gaillard vs Lyon,2012-2013,1,1,133705,133713,None,2012-08-24,17:45:00+00:00,French Ligue 1,4334,None
3,460161,Lyon vs Valenciennes,2012-2013,3,2,133713,133706,None,2012-09-01,14:00:00+00:00,French Ligue 1,4334,None
4,460178,Lyon vs Ajaccio,2012-2013,2,0,133713,133702,None,2012-09-16,14:00:00+00:00,French Ligue 1,4334,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,593018,Barcelona vs Lyon,2018-2019,5,1,133739,133713,None,2019-03-13,20:00:00,UEFA Champions League,4480,None
499,1032292,Lyon vs Bayern Munich,2019-2020,0,3,133713,133664,None,2020-08-19,19:00:00,UEFA Champions League,4480,Portugal
500,1012552,Man City vs Lyon,2019-2020,1,3,133613,133713,None,2020-08-15,19:00:00,UEFA Champions League,4480,Portugal
501,612971,Lyon vs Zenit St Petersburg,2019-2020,1,1,133713,134125,None,2019-09-17,17:55:00,UEFA Champions League,4480,


In [22]:
import requests
import json
import pandas as pd
import time

def add_europa_league_match_dataset_multiple_seasons(df_ligue1, start_season, end_season):
    """
    Adds europa League match information for Olympique Lyonnais to an existing dataset
    using the eventsround.php endpoint and respecting the limit of 100 requests per minute.

    Args:
        df_ligue1 (pandas.DataFrame): The DataFrame containing Ligue 1 match information.
        start_season (int): The start year of the first season (e.g., 2012).
        end_season (int): The start year of the last season (e.g., 2024).

    Returns:
        pandas.DataFrame: A DataFrame containing OL's match information in Ligue 1 and europa League,
                         or None if any of the requests failed.
    """

    all_europa_league_data = []
    ol_team_id = "133713"  # ID of Olympique Lyonnais
    europa_league_league_id = 4481
    api_key = "3"
    headers = {
        "X-API-KEY": f"{api_key}",
        "Content-Type": "application/json"
    }
    request_count = 0
    start_time = time.time()
    europa_league_rounds = [200, 150, 125, 16, 1, 2, 3, 4, 5, 6, 7, 8]

    for year in range(start_season, end_season + 1):
        season = f"{year}-{year + 1}"
        print(f"Retrieving Europa League data for the season {season}...")

        for round_number in europa_league_rounds:
            # Managing the request limit per minute
            if request_count >= 100:
                elapsed_time = time.time() - start_time
                if elapsed_time < 100:
                    sleep_time = 100 - elapsed_time
                    print(f"Request limit reached. Waiting for {sleep_time:.2f} seconds...")
                    time.sleep(sleep_time)
                request_count = 0
                start_time = time.time()

            print(f"  Retrieving data for round {round_number}...")
            url = f"https://www.thesportsdb.com/api/v1/json/3/eventsround.php?id={europa_league_league_id}&r={round_number}&s={season}"

            response = requests.get(url, headers=headers)
            request_count += 1

            if response.status_code == 200:
                data = response.json()
                if "events" in data and data["events"]:
                    events = data["events"]
                    for event in events:
                        # Filter OL matches
                        if event["idHomeTeam"] == ol_team_id or event["idAwayTeam"] == ol_team_id:
                            all_europa_league_data.append({
                                "idEvent": event["idEvent"],
                                "strEvent": event["strEvent"],
                                "strSeason": event["strSeason"],
                                "intHomeScore": event.get("intHomeScore"),
                                "intAwayScore": event.get("intAwayScore"),
                                "idHomeTeam": event["idHomeTeam"],
                                "idAwayTeam": event["idAwayTeam"],
                                "strTimestamp": event.get("strTimestamp"),
                                "dateEvent": event["dateEvent"],
                                "strTime": event["strTime"],
                                "strLeague": event["strLeague"],
                                "idLeague": event["idLeague"],
                                "strCountry": event.get("strCountry"),
                            })
                else:
                    print(f"    No events found for the Europa League, season {season}, and round {round_number}.")
            else:
                print(f"    Request failed with status code: {response.status_code} for season {season}, round {round_number}")
                return None  # Stop the function if a request fails

    df_europa_league = pd.DataFrame(all_europa_league_data)

    if df_europa_league.empty:
        print("No Europa League matches found for OL during the specified seasons.")
        return df_ligue1  # Return the Ligue 1 DataFrame if no Europa League matches are found
    else:
        # Combine the DataFrames
        combined_df = pd.concat([df_ligue1, df_europa_league], ignore_index=True)
        return combined_df
    
# Exemple d'utilisation (en supposant que vous avez déjà exécuté la première fonction):
start_season = 2009
end_season = 2024
df_ligue1 = add_europa_league_match_dataset_multiple_seasons(df_ligue1, start_season, end_season)

Retrieving Europa League data for the season 2009-2010...
  Retrieving data for round 200...
  Retrieving data for round 150...
    No events found for the Europa League, season 2009-2010, and round 150.
  Retrieving data for round 125...
    No events found for the Europa League, season 2009-2010, and round 125.
  Retrieving data for round 16...
    No events found for the Europa League, season 2009-2010, and round 16.
  Retrieving data for round 1...
    No events found for the Europa League, season 2009-2010, and round 1.
  Retrieving data for round 2...
    No events found for the Europa League, season 2009-2010, and round 2.
  Retrieving data for round 3...
    No events found for the Europa League, season 2009-2010, and round 3.
  Retrieving data for round 4...
    No events found for the Europa League, season 2009-2010, and round 4.
  Retrieving data for round 5...
    No events found for the Europa League, season 2009-2010, and round 5.
  Retrieving data for round 6...
    No e

In [23]:
df_ligue1.head(100000)

,idEvent,strEvent,strSeason,intHomeScore,intAwayScore,idHomeTeam,idAwayTeam,strTimestamp,dateEvent,strTime,strLeague,idLeague,strCountry
0,460132,Rennes vs Lyon,2012-2013,0,1,133719,133713,None,2012-08-11,18:00:00+00:00,French Ligue 1,4334,None
1,460141,Lyon vs Troyes,2012-2013,4,1,133713,134789,None,2012-08-18,14:00:00+00:00,French Ligue 1,4334,None
2,460150,Evian Thonon Gaillard vs Lyon,2012-2013,1,1,133705,133713,None,2012-08-24,17:45:00+00:00,French Ligue 1,4334,None
3,460161,Lyon vs Valenciennes,2012-2013,3,2,133713,133706,None,2012-09-01,14:00:00+00:00,French Ligue 1,4334,None
4,460178,Lyon vs Ajaccio,2012-2013,2,0,133713,133702,None,2012-09-16,14:00:00+00:00,French Ligue 1,4334,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,2150673,Hoffenheim vs Lyon,2024-2025,2,2,133657,133713,2024-11-07T20:00:00,2024-11-07,20:00:00,UEFA Europa League,4481,Germany
517,2150680,Qarabag FK vs Lyon,2024-2025,1,4,135702,133713,2024-11-28T17:45:00,2024-11-28,17:45:00,UEFA Europa League,4481,Azerbaijan
518,2150706,Lyon vs Eintracht Frankfurt,2024-2025,3,2,133713,133814,2024-12-12T20:00:00,2024-12-12,20:00:00,UEFA Europa League,4481,France
519,2150716,Fenerbahce vs Lyon,2024-2025,None,None,133807,133713,2025-01-23T17:45:00,2025-01-23,17:45:00,UEFA Europa League,4481,Türkiye


In [24]:
import pandas as pd

df.to_pickle('raw_data_ol_match.pkl')
# df = pd.read_pickle('raw_data_ol.pkl')